---
subject: Programmation
venue: OCaml
title: "Présentation de xeus-ocaml"
subtitle: Programmer en OCaml avec Jupyterlite
authors:
  - name: Davy Cottet
    affiliations:
      - Lycée Expérimental
license: CC-BY-NC-4.0
---


# Bienvenue dans xeus-ocaml ! 🚀

`xeus-ocaml` est un noyau Jupyter pour le langage de programmation OCaml qui s'exécute entièrement dans votre navigateur web, grâce à WebAssembly.

Ce notebook vous présentera ses fonctionnalités clés :
- **Toplevel OCaml interactif** : Exécutez du code OCaml et visualisez les résultats instantanément.
- **Assistance au code** : Profitez de la complétion de code et de l'inspection des types/documentation, grâce à Merlin.
- **Affichage riche** : Générez du HTML, du Markdown, des graphiques et plus encore, directement depuis OCaml.
- **Chargement dynamique de bibliothèques** : Utilisez la directive `#require` pour charger des bibliothèques externes comme `ocamlgraph`.
- **Système de fichiers virtuel** : Manipulez des fichiers en utilisant les fonctions d'entrée/sortie standards d'OCaml.

Commençons !

## 1. Exécution de base du Toplevel

Vous pouvez exécuter du code OCaml comme dans un toplevel classique. Chaque phrase doit se terminer par `;;`.

In [ ]:
1 + 1;;

"Hello, " ^ "Jupyter!";;

Le noyau conserve son état entre les cellules. Vous pouvez définir une fonction dans une cellule et l'utiliser dans une autre.

In [ ]:
let greet name = "Hello, " ^ name ^ "!";;

In [ ]:
greet "xeus-ocaml";;


## 2. Assistance au code avec Merlin

`xeus-ocaml` intègre Merlin pour offrir une expérience similaire à celle d'un EDI.

#### Complétion de code
Dans la cellule ci-dessous, placez votre curseur après `List.` et appuyez sur `Tab` pour voir la liste des fonctions disponibles.


In [ ]:
List.

#### Inspection du code

Vous pouvez afficher le type et la documentation d'un identifiant en plaçant votre curseur dessus et en appuyant sur `Maj+Tab`. Vous pouvez également ouvrir l'aide avec `Ctrl+I`.

In [ ]:
List.map

## 3. Affichage riche avec `Xlib`

Le module `Xlib` est automatiquement ouvert au démarrage, fournissant des fonctions pour générer des affichages riches comme du HTML, du Markdown, et même des graphiques interactifs.


In [ ]:
output_html "<h1>Ceci est un en-tête HTML</h1><p>Généré depuis OCaml !</p>";;

output_markdown "# Et ceci est du Markdown\n\n* Avec une liste à puces !\n* Et une autre.";;


Vous pouvez également afficher des données structurées comme du JSON et créer des graphiques interactifs avec Vega-Lite.

In [ ]:
(* Affiche un objet JSON *)
let json_string = "{\"name\": \"xeus-ocaml\", \"is_awesome\": true}";;
output_json json_string;;

(* Affiche un diagramme en barres Vega-Lite *)
let vega_spec = {|
  {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "description": "Un simple diagramme en barres.",
    "data": {
      "values": [
        {"a": "A", "b": 28}, {"a": "B", "b": 55}, {"a": "C", "b": 43},
        {"a": "D", "b": 91}, {"a": "E", "b": 81}, {"a": "F", "b": 53}
      ]
    },
    "mark": "bar",
    "encoding": {
      "x": {"field": "a", "type": "nominal"},
      "y": {"field": "b", "type": "quantitative"}
    }
  }
|};;
output_vegalite vega_spec;;


## 4. Chargement dynamique de bibliothèques avec `ocamlgraph`

Vous pouvez charger des bibliothèques externes en utilisant la directive `#require`. Chargeons `ocamlgraph` pour construire et visualiser un graphe.


In [ ]:
#require "ocamlgraph";;

### Création d'un graphe simple

Commençons par définir un module pour un graphe simple, persistant et orienté.
- **`Persistent`** : La structure de données du graphe est immuable. Chaque opération (comme l'ajout d'une arête) renvoie un *nouveau* graphe.
- **`Digraph`** : Le graphe est orienté (les arêtes ont une source et une destination).
- **`Concrete`** : Les sommets eux-mêmes sont les étiquettes (dans ce cas, des chaînes de caractères `string`).


In [ ]:
open Graph;;

module G = Persistent.Digraph.Concrete(
  struct
    type t = string
    let compare = String.compare
    let hash = Hashtbl.hash
    let equal = (=)
  end
);;

(* Crée un graphe vide *)
let g = G.empty;;

print_endline "Graphe vide créé.";;

### Ajout de sommets et d'arêtes

Comme notre graphe est persistant, nous réaffectons la variable `g` avec le nouveau graphe retourné par chaque opération. Nous allons modéliser un graphe simple de vols entre des villes.

In [ ]:
let g = G.add_vertex g "Paris";;
let g = G.add_vertex g "Londres";;
let g = G.add_vertex g "New York";;
let g = G.add_vertex g "Tokyo";;

let g = G.add_edge g "Londres" "Paris";;
let g = G.add_edge g "Paris" "New York";;
let g = G.add_edge g "New York" "Tokyo";;
let g = G.add_edge g "Tokyo" "Londres";; (* Un cycle ! *)
let g = G.add_edge g "Paris" "Tokyo";;

print_endline "4 sommets et 5 arêtes ajoutés.";;

### Inspection du graphe

Nous pouvons maintenant interroger le graphe pour obtenir des informations de base comme le nombre de sommets et d'arêtes, et itérer sur ceux-ci.

In [ ]:
Printf.printf "Nombre de sommets : %d\n" (G.nb_vertex g);;
Printf.printf "Nombre d'arêtes : %d\n" (G.nb_edges g);;

print_endline "\nSommets :";;
G.iter_vertex (fun v -> print_endline v) g;;

print_endline "\nArêtes :";;
G.iter_edges_e (fun (src, dst) -> Printf.printf "%s -> %s\n" src dst) g;;

### Exécution d'un algorithme simple : Parcours en profondeur (DFS)

OCamlGraph est livré avec de nombreux algorithmes prédéfinis. Utilisons un parcours en profondeur (Depth-First Search). Nous créons un nouveau module `Dfs` en appliquant le foncteur `Graph.Traverse.Dfs` à notre module de graphe `G`.

In [ ]:
module Dfs = Traverse.Dfs(G);;

#### Ordre de parcours DFS (préfixe) :

In [ ]:
Dfs.iter ~pre:(fun v -> Printf.printf "%s " v) g;;

#### Vérification des cycles

In [ ]:
if Dfs.has_cycle g then
  print_endline "\nLe graphe contient au moins un cycle."
else
  print_endline "\nLe graphe est un DAG (Graphe Acyclique Orienté).";;

### Visualisation avec Graphviz (format Dot)

OCamlGraph peut générer une sortie au format **DOT**, qui peut être utilisée par la suite d'outils Graphviz pour produire une image du graphe.

D'abord, nous créons un module `Dot` configuré pour notre graphe `G`, en spécifiant comment les sommets et le graphe dans son ensemble doivent être stylisés.

In [ ]:
module Dot = Graphviz.Dot(
  struct
    include G (* Utilise notre module de graphe `G` *)

    (* Attributs au niveau du graphe *)
    let graph_attributes _ = [`Rankdir `LeftToRight]
    
    (* Attributs par défaut des sommets *)
    let default_vertex_attributes _ = []
    
    (* Attributs spécifiques aux sommets *)
    let vertex_name v = Printf.sprintf "\"%s\"" v (* Met les noms des sommets entre guillemets *)
    let vertex_attributes _ = [`Shape `Box; `Style `Rounded]
    
    (* Attributs par défaut des arêtes *)
    let default_edge_attributes _ = []
    
    (* Attributs spécifiques aux arêtes *)
    let edge_attributes _ = []
    
    (* Gestion des sous-graphes (non utilisée ici) *)
    let get_subgraph _ = None
  end
);;

### Génération de la chaîne de caractères dot du graphe (graphviz)

Maintenant, générons la représentation DOT et affichons-la !

In [ ]:
let dot_string = Format.asprintf "%a" Dot.fprint_graph g

### Visualisation SVG avec l'extension intégrée et viz.js

In [ ]:
output_dot dot_string

## 5. Système de fichiers virtuel

Le noyau inclut un système de fichiers virtuel en mémoire. Vous pouvez utiliser les fonctions standards d'OCaml `In_channel`, `Out_channel`, et `Sys` pour interagir avec lui.

In [ ]:
(* Écrit un message dans un fichier *)
let message = "Bonjour depuis le système de fichiers virtuel !";;
let oc = open_out "hello.txt";;
output_string oc message;;
close_out oc;;

In [ ]:
(* Relit le message *)
let ic = open_in "hello.txt";;
let read_message = input_line ic;;
close_in ic;;

read_message;;

Nous pouvons vérifier que le fichier a été créé en listant le contenu du répertoire courant.

In [ ]:
Sys.readdir ".";;

## 🎉 Conclusion

Ce notebook a démontré les fonctionnalités principales de `xeus-ocaml`. Vous avez vu comment exécuter du code, obtenir une assistance linguistique riche, afficher des visualisations, charger des bibliothèques et interagir avec un système de fichiers virtuel, le tout depuis votre navigateur.

Pour plus d'informations, visitez le [dépôt du projet sur GitHub](https://github.com/davy39/xeus-ocaml).